In [ ]:
%pylab inline
import scipy
import h5py
import skimage
import os
from skimage import io,transform,img_as_float
from skimage.io import imread,imsave
from collections import OrderedDict
import decimal
notebook_dir = os.getcwd()
project_dir = os.path.split(notebook_dir)[0]
result_dir = project_dir + '/Results/Images/'
if not os.path.isdir(result_dir):
    os.makedirs(result_dir)
tmp_dir = project_dir + '/Tmp/'
if not os.path.isdir(tmp_dir):
    os.makedirs(tmp_dir)
photo_dir = project_dir + '/Images/ControlPaper/'
art_dir = project_dir + '/Images/ControlPaper/'

In [ ]:
#functions to save and load hdf5 files to pass data to torch
def make_torch_input(filename, layers, loss_functions, args):
    f = h5py.File(filename,'w')
    for l,layer in enumerate(layers):
        layer_group = f.create_group(layer)
        for lf,loss_function in enumerate(loss_functions[l]):
            lf_group = layer_group.create_group(loss_function)
            for arg in args[l][lf]:
                dataset = lf_group.create_dataset(arg, data=args[l][lf][arg])
    f.close()
    
def make_torch_init(filename, init):
    f = h5py.File(filename,'w')
    f.create_dataset('init', data=init)
    f.close()

def get_torch_output(filename):
    f = h5py.File(filename,'r')
    data = f['opt_result']
    return data.value
    f.close()
    
def get_torch_loss(filename):
    f = h5py.File(filename,'r')
    data = f['losses']
    return data.value
    f.close()

def list2css(layers):
    '''
    Takes list of strings and returns comma separated string
    '''
    css = str()
    for l in layers:
        css = css+l+','
    return css[:-1]

def get_activations(images, caffe_model, layers='all', gpu=0):
    '''
    Function to get neural network activations in response to images from torch.
    
    :param images: array of images
    :param caffe_model: file name of the network .caffemodel file
    :param layers: network layers for which the activations should be computed
    :return: network activations in response to images
    '''
    layers = list2css(layers)
    images_file_name = tmp_dir + 'images.hdf5'
    output_file_name = tmp_dir + 'activations.hdf5'
    f = h5py.File(images_file_name, 'w')
    f.create_dataset('images', data=images)
    f.close()
    context = {
    'caffe_model': caffe_model,
    'images': images_file_name,
    'layers': layers,
    'gpu': gpu,
    'backend': 'cudnn',
    'output_file': output_file_name,
    'project_dir': project_dir
    }
    template = ('#!/bin/bash\n' +
                'cd {project_dir} && ' + 
                '/usr/local/torch/install/bin/th ComputeActivations.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-images {images} ' + 
                '-layers {layers} ' + 
                '-gpu {gpu} ' + 
                '-backend {backend} ' +
                '-output_file {output_file}')
    script_name = project_dir + '/get_activations.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    os.chmod(script_name, 0o755)
    #execute script 
    !{script_name} >/dev/null
    f = h5py.File(output_file_name,'r')
    act = OrderedDict()
    for key in f.keys():
        act[key] = f[key].value.copy()
    f.close()
    return act

def preprocess(image):
    assert(image.max() <= 1.001)
    imagenet_mean = array([0.40760392,  0.45795686,  0.48501961])
    image_torch = 255 * (image[:,:,::-1] - imagenet_mean).transpose(2,0,1)
    return image_torch

def deprocess(image_torch):
    imagenet_mean = array([0.40760392,  0.45795686,  0.48501961])
    image = (image_torch.transpose(1,2,0)/255. + imagenet_mean)[:,:,::-1]
    image[image>1] = 1
    image[image<0] = 0
    return image

def gram_matrix(activations):
    n_fm = activations.shape[0]
    F = activations.reshape(n_fm,-1)
    G = F.dot(F.T) / F[0,:].size
    return G

import itertools
def flatten(l):
    return list(itertools.chain.from_iterable(l))

def set_model(name, project_dir):
    if name == 'org_pad':
        model = project_dir + '/Models/VGG_ILSVRC_19_layers_conv.caffemodel'
    elif name == 'norm_pad':
        model = project_dir + '/Models/vgg_normalised.caffemodel'
    else:
        assert False, 'unknown model name'
    return model
    
def match_color(target_img, source_img, mode='pca', eps=1e-5):
    '''
    Matches the colour distribution of the target image to that of the source image
    using a linear transform.
    Images are expected to be of form (w,h,c) and float in [0,1].
    Modes are chol, pca or sym for different choices of basis.
    '''
    mu_t = target_img.mean(0).mean(0)
    t = target_img - mu_t
    t = t.transpose(2,0,1).reshape(3,-1)
    Ct = t.dot(t.T) / t.shape[1] + eps * eye(t.shape[0])
    mu_s = source_img.mean(0).mean(0)
    s = source_img - mu_s
    s = s.transpose(2,0,1).reshape(3,-1)
    Cs = s.dot(s.T) / s.shape[1] + eps * eye(s.shape[0])
    if mode == 'chol':
        chol_t = np.linalg.cholesky(Ct)
        chol_s = np.linalg.cholesky(Cs)
        ts = chol_s.dot(np.linalg.inv(chol_t)).dot(t)
    if mode == 'pca':
        eva_t, eve_t = np.linalg.eigh(Ct)
        Qt = eve_t.dot(np.sqrt(np.diag(eva_t))).dot(eve_t.T)
        eva_s, eve_s = np.linalg.eigh(Cs)
        Qs = eve_s.dot(np.sqrt(np.diag(eva_s))).dot(eve_s.T)
        ts = Qs.dot(np.linalg.inv(Qt)).dot(t)
    if mode == 'sym':
        eva_t, eve_t = np.linalg.eigh(Ct)
        Qt = eve_t.dot(np.sqrt(np.diag(eva_t))).dot(eve_t.T)
        Qt_Cs_Qt = Qt.dot(Cs).dot(Qt)
        eva_QtCsQt, eve_QtCsQt = np.linalg.eigh(Qt_Cs_Qt)
        QtCsQt = eve_QtCsQt.dot(np.sqrt(np.diag(eva_QtCsQt))).dot(eve_QtCsQt.T)
        ts = np.linalg.inv(Qt).dot(QtCsQt).dot(np.linalg.inv(Qt)).dot(t)
    matched_img = ts.reshape(*target_img.transpose(2,0,1).shape).transpose(1,2,0)
    matched_img += mu_s
    matched_img[matched_img>1] = 1
    matched_img[matched_img<0] = 0
    return matched_img

In [ ]:
def transfer_texture(transfer_img, style_img, project_dir, layers, weights,
                     model_name='org_pad', gpu=0, max_iter=500):
    '''
    Initialises from transfer_img and transfers texture from style_img on the layers that are specified
    '''
    assert len(layers)==len(weights), 'need as many weights as layers'
    caffe_model = set_model(model_name, project_dir)
    input_file_name = tmp_dir + 'input.hdf5'
    init_file_name = tmp_dir + 'init.hdf5'
    output_file_name = tmp_dir + 'output.hdf'
    loss_functions = [['GramMSE']]*len(layers)
    act = OrderedDict()
    act['style'] = get_activations(style_img, caffe_model, layers=layers, gpu=gpu)
    args = [
    [
        {'targets': gram_matrix(act['style'][layer])[None,:],
          'weights': weights[l][0]}
    ] for l,layer in enumerate(layers)
    ]
    make_torch_input(input_file_name, layers, loss_functions, args)
    make_torch_init(init_file_name, transfer_img)
    context = {
        'caffe_model': caffe_model,
        'input_file': input_file_name,
        'init_file': init_file_name,
        'gpu': gpu,
        'max_iter': max_iter,
        'backend': 'cudnn',
        'print_iter': 250,
        'save_iter': 0,
        'layer_order': list2css(layers),
        'output_file': output_file_name,
        'project_dir': project_dir
    }
    template = (
                '#!/bin/bash\n' +
                'cd {project_dir} && ' + 
                'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-input_file {input_file} ' + 
                '-init_file {init_file} ' + 
                '-gpu {gpu} ' + 
                '-max_iter {max_iter} ' +
                '-print_iter {print_iter} ' +
                '-save_iter {save_iter} ' +
                '-backend {backend} ' + 
                '-layer_order {layer_order} ' +
                '-output_file {output_file} '
               )
    script_name = project_dir + '/run_synthesis.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    os.chmod(script_name, 0o755)
    #execute script
    !{script_name}
    output = get_torch_output(output_file_name)
    
    return output

## Combine 2 images with fine and coarse scale

### Use original VGG-19 network

In [ ]:
img_dirs = OrderedDict()
img_dirs['content'] = photo_dir
img_dirs['style_color'] = photo_dir
img_dirs['style_pretrans'] = art_dir
img_dirs['style'] = art_dir
img_names = OrderedDict()
img_names['content'] = 'fig4_content.jpg'
img_names['style_color'] = 'fig4_content.jpg' #image from which to use the colour map
img_names['style_pretrans'] = 'fig4_style2.jpg' #fine scale style image
img_names['style'] = 'fig4_style3.jpg' #coarse scale style image
layers = OrderedDict()
layers['content'] =  ['relu4_2']
layers['style'] = ['relu1_1','relu2_1','relu3_1','relu4_1','relu5_1']
layers_now = layers['style'] + layers['content']
loss_functions = [['GramMSE']] * len(layers['style']) + [['MSE']]
img_size = 512
max_iter = 500
hr_img_size = 1024
hr_max_iter = 200
gpu = 0
pretrans_color = True; #
max_pretrans_layer = 'relu2_1'
hr_max_pretrans_layer = 'relu4_1'
assert(max_pretrans_layer in layers['style'])
model_name = 'org_pad'
caffe_model = set_model(model_name, project_dir)
input_file_name = tmp_dir + 'input.hdf5'
init_file_name = tmp_dir + 'init.hdf5'
output_file_name = tmp_dir + 'output.hdf'
sw = 1e3
cw = 1
ptw = 1e3
weights = OrderedDict()
weights['style'] = [
    [array([sw/64**2])],
    [array([sw/128**2])],
    [array([sw/256**2])],
    [array([sw/512**2])],
    [array([sw/512**2])],
]
weights['content'] = [[array([cw])]]
result_image_name = (
'cimg_cm_' + img_names['content'] + '_' +
'scimg_' + img_names['style_color'] +
'_spimg_' + img_names['style_pretrans'] +
'_simg_' + img_names['style'] +
'_pt_layer_' + max_pretrans_layer + 
'_sz_' + str(img_size) + 
'_model_' + model_name + 
'_sw_' + '%.1E'%decimal.Decimal(sw) + 
'_cw_' + '%.1E'%decimal.Decimal(cw) + 
'.jpg'
)
pretrans_image_name = (
'spimg_' + img_names['style_pretrans'] +
'_simg_' + img_names['style'] +
'_pt_layer_' + max_pretrans_layer + 
'_sz_' + str(img_size) + 
'_model_' + model_name + 
'_ptw_' + '%.1E'%decimal.Decimal(ptw) + 
'.jpg'
)

In [ ]:
#check if image exists
if os.path.isfile(result_dir + result_image_name) == False:
    #get images
    conditions = img_names.keys()
    imgs = OrderedDict()
    imgs_torch = OrderedDict()
    act = OrderedDict()
    for cond in conditions:
        imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
        try:
            imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / img_size**2))
        except:
            print('no downsampling: ' + img_names[cond])
        imshow(imgs[cond]);show()
        imgs_torch[cond] = preprocess(imgs[cond])
    #transfer color and pretrans style onto style image
    if pretrans_color:
        #transfer color onto later style images
        t = False
        for key in imgs.keys():
            if t:
                print(key, 'style_color')
                imgs[key] = match_color(imgs[key], imgs['style_color'], mode='pca')
                imgs_torch[key] = preprocess(imgs[key])
            if key == 'style_color':
                t = True        
    if max_pretrans_layer =='none':
        #only use style
        for key in imgs.keys():
            imshow(imgs[key], interpolation='nearest');show()
    elif max_pretrans_layer =='all':
        #only use pretrans style
        imgs_torch['style'] = imgs_torch['style_pretrans']
        for key in imgs.keys():
            imshow(imgs[key], interpolation='nearest');show()
    else:  
        #transfer pretrans style onto later style image
        pretrans_layers = layers['style'][:layers['style'].index(max_pretrans_layer)+1]
        pretrans_weights = [[array([ptw/len(pretrans_layers)])]]*len(pretrans_layers)
        t = False
        for key in imgs.keys():
            if t:
                print(key,'style_pretrans')
                imgs_torch[key] = transfer_texture(imgs_torch[key],imgs_torch['style_pretrans'], project_dir,
                                                   pretrans_layers, pretrans_weights,
                                                   gpu=gpu, model_name=model_name)
                imgs[key] = deprocess(imgs_torch[key])
            if key == 'style_pretrans':
                t = True
        for key in imgs.keys():
            imshow(imgs[key], interpolation='nearest');show()
    #save pretransformed style image
    imsave(result_dir + pretrans_image_name, imgs['style'])
    #compute activations
    act = OrderedDict()
    act['content'] = get_activations(imgs_torch['content'], caffe_model, layers=layers['content'],gpu=gpu)
    act['style'] = get_activations(imgs_torch['style'], caffe_model, layers=layers['style'],gpu=gpu)
    #set arguments and save input and init files
    args = OrderedDict()
    args['style'] = [
                        [
                            {'targets': gram_matrix(act['style'][layer])[None,:],
                             'weights': weights['style'][l][0]} 
                        ] 
                    for l,layer in enumerate(layers['style'])]
    args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
    args_now = args['style'] + args['content']
    make_torch_input(input_file_name, layers_now, loss_functions, args_now)
    make_torch_init(init_file_name, imgs_torch['content'])
    #run image synthesis
    max_iter = max_iter
    context = {
        'caffe_model': caffe_model,
        'input_file': input_file_name,
        'init_file': init_file_name,
        'gpu': gpu,
        'max_iter': max_iter,
        'backend': 'cudnn',
        'print_iter': 50,
        'save_iter': 0,
        'layer_order': list2css(layers_now),
        'output_file': output_file_name,
        'project_dir': project_dir
        }
    template = ('#!/bin/bash\n' +
                'cd {project_dir} && ' + 
                'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-input_file {input_file} ' + 
                '-init_file {init_file} ' + 
                '-gpu {gpu} ' + 
                '-max_iter {max_iter} ' +
                '-print_iter {print_iter} ' +
                '-save_iter {save_iter} ' +
                '-backend {backend} ' + 
                '-layer_order {layer_order} ' +
                '-output_file {output_file} '
               )

    script_name = project_dir + '/run_synthesis.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    os.chmod(script_name, 0o755)
    #execute script
    !{script_name}
    output = deprocess(get_torch_output(output_file_name))
    imshow(output,interpolation='nearest');show()
    #save image
    imsave(result_dir + result_image_name, output)  
#make high res
if hr_img_size:
    assert(hr_max_pretrans_layer in layers['style'])
    #load low-res result
    lr_output = img_as_float(imread(result_dir + result_image_name))
    #define new output file names
    result_image_name = (
    'cimg_cm_' + img_names['content'] + '_' +
    'scimg_' + img_names['style_color'] +
    '_spimg_' + img_names['style_pretrans'] +
    '_simg_' + img_names['style'] +
    '_pt_layer_' + max_pretrans_layer + 
    '_hrpt_layer_' + hr_max_pretrans_layer + 
    '_sz_' + str(img_size) + 
    '_hrsz_' + str(hr_img_size) + 
    '_model_' + model_name + 
    '_sw_' + '%.1E'%decimal.Decimal(sw) + 
    '_cw_' + '%.1E'%decimal.Decimal(cw) + 
    '.jpg'
    )
    pretrans_image_name = (
    'spimg_' + img_names['style_pretrans'] +
    '_simg_' + img_names['style'] +
    '_hrpt_layer_' + hr_max_pretrans_layer + 
    '_hrsz_' + str(hr_img_size) + 
    '_model_' + model_name + 
    '_ptw_' + '%.1E'%decimal.Decimal(ptw) + 
    '.jpg'
    )
    #check if result exists
    if os.path.isfile(result_dir + result_image_name) == False: #check if file exists
        #get images
        conditions = img_names.keys()
        imgs = OrderedDict()
        imgs_torch = OrderedDict()
        for cond in conditions:
            imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
            try:
                imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / hr_img_size**2))
            except:
                print('no downsampling: ' + img_names[cond])
            imshow(imgs[cond]);show()
            imgs_torch[cond] = preprocess(imgs[cond])

        if pretrans_color:
            #transfer color onto later style images
            t = False
            for key in imgs.keys():
                if t:
                    print(key, 'style_color')
                    imgs[key] = match_color(imgs[key], imgs['style_color'], mode='pca')
                    imgs_torch[key] = preprocess(imgs[key])
                if key == 'style_color':
                    t = True        
        if hr_max_pretrans_layer =='none':
            #only use style
            for key in imgs.keys():
                imshow(imgs[key], interpolation='nearest');show()
        elif hr_max_pretrans_layer =='all':
            #only use pretrans style
            imgs_torch['style'] = imgs_torch['style_pretrans']
            for key in imgs.keys():
                imshow(imgs[key], interpolation='nearest');show()
        else:  
            #transfer pretrans style onto later style image
            pretrans_layers = layers['style'][:layers['style'].index(hr_max_pretrans_layer)+1]
            pretrans_weights = [[array([ptw/len(pretrans_layers)])]]*len(pretrans_layers)
            t = False
            for key in imgs.keys():
                if t:
                    print(key,'style_pretrans')
                    imgs_torch[key] = transfer_texture(imgs_torch[key],imgs_torch['style_pretrans'], project_dir,
                                                       pretrans_layers, pretrans_weights,
                                                       gpu=gpu, model_name=model_name, max_iter=200)
                    imgs[key] = deprocess(imgs_torch[key])
                if key == 'style_pretrans':
                    t = True

            for key in imgs.keys():
                imshow(imgs[key], interpolation='nearest');show()
        #save pretransformed style image
        imsave(result_dir + pretrans_image_name, imgs['style'])
        #compute activations
        act = OrderedDict()
        act['content'] = get_activations(imgs_torch['content'], caffe_model, layers=layers['content'],gpu=gpu)
        act['style'] = get_activations(imgs_torch['style'], caffe_model, layers=layers['style'],gpu=gpu)
        #set arguments and save input and init files
        args = OrderedDict()
        args['style'] = [
                            [
                                {'targets': gram_matrix(act['style'][layer])[None,:],
                                 'weights': weights['style'][l][0]} 
                            ] 
                        for l,layer in enumerate(layers['style'])]

        args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
        args_now = args['style'] + args['content']
        make_torch_input(input_file_name, layers_now, loss_functions, args_now)
        #make hr init
        hr_init = preprocess(img_as_float(scipy.misc.imresize(lr_output, imgs['content'].shape)))
        make_torch_init(init_file_name, hr_init)
        #run image synthesis
        max_iter = hr_max_iter
        context = {
            'caffe_model': caffe_model,
            'input_file': input_file_name,
            'init_file': init_file_name,
            'gpu': gpu,
            'max_iter': max_iter,
            'backend': 'cudnn',
            'print_iter': 50,
            'save_iter': 0,
            'layer_order': list2css(layers_now),
            'output_file': output_file_name,
            'project_dir': project_dir
        }
        template = ('#!/bin/bash\n' +
                    'cd {project_dir} && ' + 
                    'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                    '-caffe_model {caffe_model} ' +
                    '-input_file {input_file} ' + 
                    '-init_file {init_file} ' + 
                    '-gpu {gpu} ' + 
                    '-max_iter {max_iter} ' +
                    '-print_iter {print_iter} ' +
                    '-save_iter {save_iter} ' +
                    '-backend {backend} ' + 
                    '-layer_order {layer_order} ' +
                    '-output_file {output_file} '
                   )

        script_name = project_dir + '/run_synthesis.sh'
        with open(script_name, 'w') as script:
            script.write(template.format(**context))
        os.chmod(script_name, 0o755)
        #execute script
        !{script_name}
        output = deprocess(get_torch_output(output_file_name))
        imshow(output,interpolation='nearest');show()
        #save image
        imsave(result_dir + result_image_name, output)

### norm net

In [ ]:
img_dirs = OrderedDict()
img_dirs['content'] = photo_dir
img_dirs['style_color'] = photo_dir
img_dirs['style_pretrans'] = art_dir
img_dirs['style'] = art_dir
img_names = OrderedDict()
img_names['content'] = 'fig4_content.jpg'
img_names['style_color'] = 'fig4_content.jpg' #image from which to use the colour map
img_names['style_pretrans'] = 'fig4_style2.jpg' #fine scale style image
img_names['style'] = 'fig4_style3.jpg' #coarse scale style image
for key in img_names.keys():
    assert os.path.isfile(img_dirs[key]+img_names[key]), 'File does not exist'
layers = OrderedDict()
layers['content'] =  ['relu4_2']
layers['style'] = ['relu1_1','relu2_1','relu3_1','relu4_1','relu5_1']
layers_now = layers['style'] + layers['content']
loss_functions = [['GramMSE']] * len(layers['style']) + [['MSE']]
img_size = 512
max_iter = 500
hr_img_size = 1024
hr_max_iter = 200
gpu = 0
pretrans_color = True;
max_pretrans_layer = 'relu2_1'
hr_max_pretrans_layer = 'relu4_1'
assert(max_pretrans_layer in layers['style'])
model_name = 'norm_pad'
caffe_model = set_model(model_name, project_dir)
input_file_name = tmp_dir + 'input.hdf5'
init_file_name = tmp_dir + 'init.hdf5'
output_file_name = tmp_dir + 'output.hdf'
sw = 1e9 / len(layers['style'])
cw = 1e5
ptw = 1e5
weights = OrderedDict()
weights['style'] = [[array([sw])]] * len(layers['style'])
weights['content'] = [[array([cw])]]
result_image_name = (
'cimg_cm_' + img_names['content'] + '_' +
'scimg_' + img_names['style_color'] +
'_spimg_' + img_names['style_pretrans'] +
'_simg_' + img_names['style'] +
'_pt_layer_' + max_pretrans_layer + 
'_sz_' + str(img_size) + 
'_model_' + model_name + 
'_sw_' + '%.1E'%decimal.Decimal(sw) + 
'_cw_' + '%.1E'%decimal.Decimal(cw) + 
'.jpg'
)
pretrans_image_name = (
'scimg_' + img_names['style_color'] +
'_spimg_' + img_names['style_pretrans'] +
'_simg_' + img_names['style'] +
'_pt_layer_' + max_pretrans_layer + 
'_sz_' + str(img_size) + 
'_model_' + model_name + 
'_ptw_' + '%.1E'%decimal.Decimal(ptw) + 
'.jpg'
)

In [ ]:
#check if image exists
if os.path.isfile(result_dir + result_image_name) == False:
    #get images
    conditions = img_names.keys()
    imgs = OrderedDict()
    imgs_torch = OrderedDict()
    act = OrderedDict()
    for cond in conditions:
        imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
        try:
            imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / img_size**2))
        except:
            print('no downsampling: ' + img_names[cond])
        imshow(imgs[cond]);show()
        imgs_torch[cond] = preprocess(imgs[cond])
    #transfer color and pretrans style onto style image
    if pretrans_color:
        #transfer color onto later style images
        t = False
        for key in imgs.keys():
            if t:
                print(key, 'style_color')
                imgs[key] = match_color(imgs[key], imgs['style_color'], mode='pca')
                imgs_torch[key] = preprocess(imgs[key])
            if key == 'style_color':
                t = True        
    if max_pretrans_layer =='none':
        #only use style
        for key in imgs.keys():
            imshow(imgs[key], interpolation='nearest');show()
    elif max_pretrans_layer =='all':
        #only use pretrans style
        imgs_torch['style'] = imgs_torch['style_pretrans']
        for key in imgs.keys():
            imshow(imgs[key], interpolation='nearest');show()
    else:  
        #transfer pretrans style onto later style image
        pretrans_layers = layers['style'][:layers['style'].index(max_pretrans_layer)+1]
        pretrans_weights = [[array([ptw/len(pretrans_layers)])]]*len(pretrans_layers)
        t = False
        for key in imgs.keys():
            if t:
                print(key,'style_pretrans')
                imgs_torch[key] = transfer_texture(imgs_torch[key],imgs_torch['style_pretrans'], project_dir,
                                                   pretrans_layers, pretrans_weights,
                                                   gpu=gpu, model_name=model_name)
                imgs[key] = deprocess(imgs_torch[key])
            if key == 'style_pretrans':
                t = True
        for key in imgs.keys():
            imshow(imgs[key], interpolation='nearest');show()
    #save pretransformed style image
    imsave(result_dir + pretrans_image_name, imgs['style'])
    #compute activations
    act = OrderedDict()
    act['content'] = get_activations(imgs_torch['content'], caffe_model, layers=layers['content'],gpu=gpu)
    act['style'] = get_activations(imgs_torch['style'], caffe_model, layers=layers['style'],gpu=gpu)
    #set arguments and save input and init files
    args = OrderedDict()
    args['style'] = [
                        [
                            {'targets': gram_matrix(act['style'][layer])[None,:],
                             'weights': weights['style'][l][0]} 
                        ] 
                    for l,layer in enumerate(layers['style'])]

    args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
    args_now = args['style'] + args['content']
    make_torch_input(input_file_name, layers_now, loss_functions, args_now)
    make_torch_init(init_file_name, imgs_torch['content'])
    #run image synthesis
    max_iter = max_iter
    context = {
        'caffe_model': caffe_model,
        'input_file': input_file_name,
        'init_file': init_file_name,
        'gpu': gpu,
        'max_iter': max_iter,
        'backend': 'cudnn',
        'print_iter': 50,
        'save_iter': 0,
        'layer_order': list2css(layers_now),
        'output_file': output_file_name,
        'project_dir': project_dir
    }
    template = ('#!/bin/bash\n' +
                'cd {project_dir} && ' + 
                'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-input_file {input_file} ' + 
                '-init_file {init_file} ' + 
                '-gpu {gpu} ' + 
                '-max_iter {max_iter} ' +
                '-print_iter {print_iter} ' +
                '-save_iter {save_iter} ' +
                '-backend {backend} ' + 
                '-layer_order {layer_order} ' +
                '-output_file {output_file} '
               )

    script_name = project_dir + '/run_synthesis.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    os.chmod(script_name, 0o755)
    #execute script
    !{script_name}
    output = deprocess(get_torch_output(output_file_name))
    imshow(output,interpolation='nearest');show()
    #save image
    imsave(result_dir + result_image_name, output)  
#make high res
if hr_img_size:
    assert(hr_max_pretrans_layer in layers['style'])
    #load low-res result
    lr_output = img_as_float(imread(result_dir + result_image_name))
    #define new output file names
    result_image_name = (
    'cimg_cm_' + img_names['content'] + '_' +
    'scimg_' + img_names['style_color'] +
    '_spimg_' + img_names['style_pretrans'] +
    '_simg_' + img_names['style'] +
    '_pt_layer_' + max_pretrans_layer + 
    '_hrpt_layer_' + hr_max_pretrans_layer + 
    '_sz_' + str(img_size) + 
    '_hrsz_' + str(hr_img_size) + 
    '_model_' + model_name + 
    '_sw_' + '%.1E'%decimal.Decimal(sw) + 
    '_cw_' + '%.1E'%decimal.Decimal(cw) + 
    '.jpg'
    )
    pretrans_image_name = (
    'scimg_' + img_names['style_color'] +
    '_spimg_' + img_names['style_pretrans'] +
    '_simg_' + img_names['style'] +
    '_hrpt_layer_' + hr_max_pretrans_layer + 
    '_hrsz_' + str(hr_img_size) + 
    '_model_' + model_name + 
    '_ptw_' + '%.1E'%decimal.Decimal(ptw) + 
    '.jpg'
    )
    #check if result exists
    if os.path.isfile(result_dir + result_image_name) == False: #check if file exists
        #get images
        conditions = img_names.keys()
        imgs = OrderedDict()
        imgs_torch = OrderedDict()
        for cond in conditions:
            imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
            try:
                imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / hr_img_size**2))
            except:
                print('no downsampling: ' + img_names[cond])
            imshow(imgs[cond]);show()
            imgs_torch[cond] = preprocess(imgs[cond])

        if pretrans_color:
            #transfer color onto later style images
            t = False
            for key in imgs.keys():
                if t:
                    print(key, 'style_color')
                    imgs[key] = match_color(imgs[key], imgs['style_color'], mode='pca')
                    imgs_torch[key] = preprocess(imgs[key])
                if key == 'style_color':
                    t = True        
        if hr_max_pretrans_layer =='none':
            #only use style
            for key in imgs.keys():
                imshow(imgs[key], interpolation='nearest');show()
        elif hr_max_pretrans_layer =='all':
            #only use pretrans style
            imgs_torch['style'] = imgs_torch['style_pretrans']
            for key in imgs.keys():
                imshow(imgs[key], interpolation='nearest');show()
        else:  
            #transfer pretrans style onto later style image
            pretrans_layers = layers['style'][:layers['style'].index(hr_max_pretrans_layer)+1]
            pretrans_weights = [[array([ptw/len(pretrans_layers)])]]*len(pretrans_layers)
            t = False
            for key in imgs.keys():
                if t:
                    print(key,'style_pretrans')
                    imgs_torch[key] = transfer_texture(imgs_torch[key],imgs_torch['style_pretrans'], project_dir,
                                                       pretrans_layers, pretrans_weights,
                                                       gpu=gpu, model_name=model_name, max_iter=200)
                    imgs[key] = deprocess(imgs_torch[key])
                if key == 'style_pretrans':
                    t = True

            for key in imgs.keys():
                imshow(imgs[key], interpolation='nearest');show()
        #save pretransformed style image
        imsave(result_dir + pretrans_image_name, imgs['style'])
        #compute activations
        act = OrderedDict()
        act['content'] = get_activations(imgs_torch['content'], caffe_model, layers=layers['content'],gpu=gpu)
        act['style'] = get_activations(imgs_torch['style'], caffe_model, layers=layers['style'],gpu=gpu)
        #set arguments and save input and init files
        args = OrderedDict()
        args['style'] = [
                            [
                                {'targets': gram_matrix(act['style'][layer])[None,:],
                                 'weights': weights['style'][l][0]} 
                            ] 
                        for l,layer in enumerate(layers['style'])]

        args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
        args_now = args['style'] + args['content']
        make_torch_input(input_file_name, layers_now, loss_functions, args_now)
        #make hr init
        hr_init = preprocess(img_as_float(scipy.misc.imresize(lr_output, imgs['content'].shape)))
        make_torch_init(init_file_name, hr_init)
        #run image synthesis
        max_iter = hr_max_iter
        context = {
            'caffe_model': caffe_model,
            'input_file': input_file_name,
            'init_file': init_file_name,
            'gpu': gpu,
            'max_iter': max_iter,
            'backend': 'cudnn',
            'print_iter': 50,
            'save_iter': 0,
            'layer_order': list2css(layers_now),
            'output_file': output_file_name,
            'project_dir': project_dir
        }
        template = ('#!/bin/bash\n' +
                    'cd {project_dir} && ' + 
                    'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                    '-caffe_model {caffe_model} ' +
                    '-input_file {input_file} ' + 
                    '-init_file {init_file} ' + 
                    '-gpu {gpu} ' + 
                    '-max_iter {max_iter} ' +
                    '-print_iter {print_iter} ' +
                    '-save_iter {save_iter} ' +
                    '-backend {backend} ' + 
                    '-layer_order {layer_order} ' +
                    '-output_file {output_file} '
                   )

        script_name = project_dir + '/run_synthesis.sh'
        with open(script_name, 'w') as script:
            script.write(template.format(**context))
        os.chmod(script_name, 0o755)
        #execute script
        !{script_name}
        output = deprocess(get_torch_output(output_file_name))
        imshow(output,interpolation='nearest');show()
        #save image
        imsave(result_dir + result_image_name, output)

# Naive scale combination

### norm net

In [ ]:
img_dirs = OrderedDict()
img_dirs['content'] = photo_dir
img_dirs['style_color'] = photo_dir
img_dirs['style_pretrans'] = art_dir
img_dirs['style'] = art_dir
img_names = OrderedDict()
img_names['content'] = 'fig4_content.jpg'
img_names['style_color'] = 'fig4_content.jpg' #image from which to use the colour map
img_names['style_pretrans'] = 'fig4_style2.jpg' #fine scale style image
img_names['style'] = 'fig4_style3.jpg' #coarse scale style image
layers = OrderedDict()
layers['content'] =  ['relu4_2']
layers['style'] = ['relu1_1','relu2_1','relu3_1','relu4_1','relu5_1']
layers['style_pretrans'] = ['relu1_1','relu2_1','relu3_1','relu4_1','relu5_1']
layers_now = layers['style'] + layers['content']
loss_functions = [['GramMSE']] * len(layers['style']) + [['MSE']]
img_size = 512
max_iter = 500
hr_img_size = 1024
hr_max_iter = 200
gpu = 0
pretrans_color = True;
max_pretrans_layer = 'relu2_1'
hr_max_pretrans_layer = 'relu4_1'
assert(max_pretrans_layer in layers['style'])
model_name = 'norm_pad'
caffe_model = set_model(model_name, project_dir)
input_file_name = tmp_dir + 'input.hdf5'
init_file_name = tmp_dir + 'init.hdf5'
output_file_name = tmp_dir + 'output.hdf'
sw = 1e9 / len(layers['style'])
cw = 1e5
ptw = 1e5
weights = OrderedDict()
weights['style'] = [[array([sw])]] * len(layers['style'])
weights['content'] = [[array([cw])]]
result_image_name = (
'cimg_cm_' + img_names['content'] + '_' +
'scimg_' + img_names['style_color'] +
'_spimg_' + img_names['style_pretrans'] +
'_simg_' + img_names['style'] +
'_pt_layer_' + max_pretrans_layer + 
'_sz_' + str(img_size) + 
'_model_' + model_name + 
'_sw_' + '%.1E'%decimal.Decimal(sw) + 
'_cw_' + '%.1E'%decimal.Decimal(cw) + 
'_naive_scalemix.jpg'
)

In [ ]:
#check if image exists
if os.path.isfile(result_dir + result_image_name) == False:
    #get images
    conditions = img_names.keys()
    imgs = OrderedDict()
    imgs_torch = OrderedDict()
    act = OrderedDict()
    for cond in conditions:
        imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
        try:
            imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / img_size**2))
        except:
            print('no downsampling: ' + img_names[cond])
        imshow(imgs[cond]);show()
        imgs_torch[cond] = preprocess(imgs[cond])
    #transfer color and pretrans style onto style image
    if pretrans_color:
        #transfer color onto later style images
        t = False
        for key in imgs.keys():
            if t:
                print(key, 'style_color')
                imgs[key] = match_color(imgs[key], imgs['style_color'], mode='pca')
                imgs_torch[key] = preprocess(imgs[key])
            if key == 'style_color':
                t = True 
    for key in imgs.keys():
        imshow(imgs[key], interpolation='nearest');show()
    #compute activations
    act = OrderedDict()
    for cond in [cond for cond in conditions if cond != 'style_color']:
        act[cond] = get_activations(imgs_torch[cond], caffe_model, layers=layers[cond],gpu=gpu)
    #set arguments and save input and init files
    args = OrderedDict()
    args['style_pretrans'] = [
                        [
                            {'targets': gram_matrix(act['style_pretrans'][layer])[None,:],
                             'weights': weights['style'][l][0]} 
                        ] 
                    for l,layer in enumerate(layers['style'])]
    args['style'] = [
                        [
                            {'targets': gram_matrix(act['style'][layer])[None,:],
                             'weights': weights['style'][l][0]} 
                        ] 
                    for l,layer in enumerate(layers['style'])]
    args['style'][:layers['style'].index(max_pretrans_layer)+1] = args['style_pretrans'][:layers['style'].index(max_pretrans_layer)+1]
    args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
    args_now = args['style'] + args['content']
    make_torch_input(input_file_name, layers_now, loss_functions, args_now)
    make_torch_init(init_file_name, imgs_torch['content'])
    #run image synthesis
    max_iter = max_iter
    context = {
        'caffe_model': caffe_model,
        'input_file': input_file_name,
        'init_file': init_file_name,
        'gpu': gpu,
        'max_iter': max_iter,
        'backend': 'cudnn',
        'print_iter': 50,
        'save_iter': 0,
        'layer_order': list2css(layers_now),
        'output_file': output_file_name,
        'project_dir': project_dir
    }
    template = ('#!/bin/bash\n' +
                'cd {project_dir} && ' + 
                'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                '-caffe_model {caffe_model} ' +
                '-input_file {input_file} ' + 
                '-init_file {init_file} ' + 
                '-gpu {gpu} ' + 
                '-max_iter {max_iter} ' +
                '-print_iter {print_iter} ' +
                '-save_iter {save_iter} ' +
                '-backend {backend} ' + 
                '-layer_order {layer_order} ' +
                '-output_file {output_file} '
               )

    script_name = project_dir + '/run_synthesis.sh'
    with open(script_name, 'w') as script:
        script.write(template.format(**context))
    os.chmod(script_name, 0o755)
    #execute script
    !{script_name}
    output = deprocess(get_torch_output(output_file_name))
    imshow(output,interpolation='nearest');show()
    #save image
    imsave(result_dir + result_image_name, output)  
#make high res
if hr_img_size:
    assert(hr_max_pretrans_layer in layers['style'])
    #load low-res result
    lr_output = img_as_float(imread(result_dir + result_image_name))
    #define new output file names
    result_image_name = (
    'cimg_cm_' + img_names['content'] + '_' +
    'scimg_' + img_names['style_color'] +
    '_spimg_' + img_names['style_pretrans'] +
    '_simg_' + img_names['style'] +
    '_pt_layer_' + max_pretrans_layer + 
    '_sz_' + str(img_size) + 
    '_hrsz_' + str(hr_img_size) + 
    '_model_' + model_name + 
    '_sw_' + '%.1E'%decimal.Decimal(sw) + 
    '_cw_' + '%.1E'%decimal.Decimal(cw) + 
    '_naive_scalemix.jpg'
    )
    #check if result exists
    if os.path.isfile(result_dir + result_image_name) == False: #check if file exists
        #get images
        conditions = img_names.keys()
        imgs = OrderedDict()
        imgs_torch = OrderedDict()
        for cond in conditions:
            imgs[cond] = img_as_float(imread(img_dirs[cond] + img_names[cond]))
            try:
                imgs[cond] = transform.pyramid_reduce(imgs[cond], sqrt(float(imgs[cond][:,:,0].size) / hr_img_size**2))
            except:
                print('no downsampling: ' + img_names[cond])
            imshow(imgs[cond]);show()
            imgs_torch[cond] = preprocess(imgs[cond])

        if pretrans_color:
            #transfer color onto later style images
            t = False
            for key in imgs.keys():
                if t:
                    print(key, 'style_color')
                    imgs[key] = match_color(imgs[key], imgs['style_color'], mode='pca')
                    imgs_torch[key] = preprocess(imgs[key])
                if key == 'style_color':
                    t = True        
        for key in imgs.keys():
            imshow(imgs[key], interpolation='nearest');show()
        #compute activations
        act = OrderedDict()
        for cond in [cond for cond in conditions if cond != 'style_color']:
            act[cond] = get_activations(imgs_torch[cond], caffe_model, layers=layers[cond],gpu=gpu)
        #set arguments and save input and init files
        args = OrderedDict()
        args['style_pretrans'] = [
                            [
                                {'targets': gram_matrix(act['style_pretrans'][layer])[None,:],
                                 'weights': weights['style'][l][0]} 
                            ] 
                        for l,layer in enumerate(layers['style'])]
        args['style'] = [
                            [
                                {'targets': gram_matrix(act['style'][layer])[None,:],
                                 'weights': weights['style'][l][0]} 
                            ] 
                        for l,layer in enumerate(layers['style'])]
        args['style'][:layers['style'].index(max_pretrans_layer)+1] = args['style_pretrans'][:layers['style'].index(max_pretrans_layer)+1]
        args['content'] = [[{'targets': act['content'][layers['content'][0]][None,:],'weights': weights['content'][0][0]}],]                
        args_now = args['style'] + args['content']
        make_torch_input(input_file_name, layers_now, loss_functions, args_now)
        #make hr init
        hr_init = preprocess(img_as_float(scipy.misc.imresize(lr_output, imgs['content'].shape)))
        make_torch_init(init_file_name, hr_init)
        #run image synthesis
        max_iter = hr_max_iter
        context = {
            'caffe_model': caffe_model,
            'input_file': input_file_name,
            'init_file': init_file_name,
            'gpu': gpu,
            'max_iter': max_iter,
            'backend': 'cudnn',
            'print_iter': 50,
            'save_iter': 0,
            'layer_order': list2css(layers_now),
            'output_file': output_file_name,
            'project_dir': project_dir
        }
        template = ('#!/bin/bash\n' +
                    'cd {project_dir} && ' + 
                    'time /usr/local/torch/install/bin/th ImageSynthesis.lua ' + 
                    '-caffe_model {caffe_model} ' +
                    '-input_file {input_file} ' + 
                    '-init_file {init_file} ' + 
                    '-gpu {gpu} ' + 
                    '-max_iter {max_iter} ' +
                    '-print_iter {print_iter} ' +
                    '-save_iter {save_iter} ' +
                    '-backend {backend} ' + 
                    '-layer_order {layer_order} ' +
                    '-output_file {output_file} '
                   )

        script_name = project_dir + '/run_synthesis.sh'
        with open(script_name, 'w') as script:
            script.write(template.format(**context))
        os.chmod(script_name, 0o755)
        #execute script
        !{script_name}
        output = deprocess(get_torch_output(output_file_name))
        imshow(output,interpolation='nearest');show()
        #save image
        imsave(result_dir + result_image_name, output)